In [3]:
from fractalai.model import RandomDiscreteModel
from fractalai.environment import ExternalProcess, ParallelEnvironment, AtariEnvironment
from fractalai.swarm_wave import SwarmWave
from fractalai.dataset_creator import DataGenerator
import time

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# SwarmWave?

# This algorithhm is made for solving discrete Markov decision processes when a perfect model is available

In this context, if you need to generate data to train a neural network, IO is now your bottleneck.

In [5]:
name = "MsPacmanNoFrameskip-v0"
skip_frames = 0  # Number of frames to skip at the beginning
dt_mean = 5  # Apply the same action n times in average
dt_std = 3 # Repeat same action a variable number of times
min_dt = 2 # Minimum number of consecutive steps to be taken
n_samples = 100000  # Maximum number of samples allowed
reward_limit = 150000  # Stop the sampling when this score is reached
n_walkers = 100  # Maximum witdh of the tree containing al the trajectories
render_every = 1  # print statistics every n iterations.
balance = 2  # Balance exploration vs exploitation
save_data = True # Save the data generated



In [6]:
env = ParallelEnvironment(name=name,env_class=AtariEnvironment,
                          blocking=False, n_workers=8, min_dt=1)  # We will play an Atari game
model = RandomDiscreteModel(max_wakers=n_walkers,
                            n_actions=env.n_actions) # The Agent will take discrete actions at random

In [6]:
data_env = AtariEnvironment("Pong-v0", min_dt=3)


In [7]:
#%%time
s = SwarmWave(model=model, env=env, n_walkers=n_walkers, reward_limit=reward_limit, samples_limit=n_samples,
                  render_every=render_every, balance=balance, save_data=save_data,
                  dt_mean=dt_mean, dt_std=dt_std, accumulate_rewards=True, min_dt=min_dt,
                  prune_tree=True)
#mo


In [8]:
for i in range(100):
    s.save_run(folder="early_game", print_swarm=True)

TypeError: append_leaf() missing 3 required positional arguments: 'obs', 'reward', and 'terminal'

In [ ]:
asfgafgsfdg

In [6]:
%%timeit
s.reset()
s.tree.reset()
s.run_swarm(print_swarm=False)

11 s ± 766 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
s.reset()
s.tree.reset()
s.run_swarm(print_swarm=True)

Environment: MsPacmanNoFrameskip-v0 | Walkers: 100 | Deaths: 0 | data_size 168
Total samples: 45872 Progress: 45.87%
Reward: mean 204.30 | Dispersion: 30.00 | max 220.00 | min 190.00 | std 6.21
Episode length: mean 565.21 | Dispersion 41.00 | max 580.00 | min 539.00 | std 7.76
Dt: mean 5.31 | Dispersion: 11.00 | max 13.00 | min 2.00 | std 2.76
Status: Playing...
Efficiency 1.16%
Generated 531 Examples | 86.39 samples per example.



In [11]:
s.render_game(sleep=0.01)

In [ ]:
print(s)

In [ ]:
s.tree.process_stacking()

In [ ]:
s.tree.dataset.nodes

In [ ]:
sum([s.tree.data.edges[e]["dt"] for e in s.tree.data.edges])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
obs_0, action, reward, new_obs, end = s.tree.node_to_example("198770_1")
obs_1, action, reward, new_obs, end = s.tree.node_to_example("198770_1")
plt.imshow(obs_0[:,:, 0])

In [ ]:
plt.imshow(obs_1[:,:, 2])

In [ ]:

plt.imshow(obs_0[:,:, 0] + obs_1[:,:, 0])

In [ ]:
plt.imshow(obs_1[:,:, 0])

In [ ]:
plt.imshow(obs_1[:,:, 1])

In [ ]:
plt.imshow(obs_1[:,:, 2])

In [ ]:
"pene

In [ ]:
from fractalai.fdqn import FDQN

In [ ]:
fqn = FDQN(s)

In [ ]:
while True:
    fqn.train_network(2, False)

In [ ]:
from collections import deque

In [ ]:
d = deque(maxlen=100)

In [ ]:
d.append(10)

In [ ]:
[1 for i in range(0)]

In [ ]:
fdqn.test_network()

In [ ]:
gen = s.batch_generator(batch_size=-1)

In [ ]:
val = next(gen)

In [ ]:
len(val[2])

In [ ]:
for val in s.example_generator():
    print(val)

In [ ]:
_, obs = data_env.reset()

In [ ]:
plt.imshow(obs[32:196, 5:155])

In [ ]:
import cv2

In [ ]:
s.tree.dataset.node

In [ ]:
frame = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(s.tree.dataset.nodes.get("14781_0")["new_obs"][0,:,:,0])

In [ ]:
plt.imshow(s.tree.dataset.nodes.get("25066_1")["new_obs"])

In [ ]:
s.tree.data.node[13]["state"]

In [ ]:
s.render_game()

In [ ]:
_ ,obs = data_env.reset()

In [ ]:
plt.imshow(obs[25:195])

In [ ]:
obs.shape

In [ ]:
sess = tf.Session()
K.set_session(sess)
env = gym.make("Pendulum-v0")
actor_critic = ActorCritic(env, sess, input_shape=(32,32,4))

In [ ]:
actor_critic.train()